<a href="https://colab.research.google.com/github/Keveen-ghori/Question-Answering-System-using-Haystack/blob/main/qna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Make sure you have a GPU running
!nvidia-smi

Fri May 13 04:06:14 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from haystack.document_store.elasticsearch import ElasticsearchDocumentStore

ModuleNotFoundError: ignored

INFO - haystack.modeling.model.optimization -  apex not found, won't use it. See https://nvidia.github.io/apex/
ERROR - root -  Failed to import 'magic' (from 'python-magic' and 'python-magic-bin' on Windows). FileTypeClassifier will not perform mimetype detection on extensionless files. Please make sure the necessary OS libraries are installed if you need this functionality.


In [ ]:
# In-Memory Document Store
from haystack.document_stores import InMemoryDocumentStore

document_store = InMemoryDocumentStore()

INFO - haystack.telemetry -  Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by calling disable_telemetry() or by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems on the documentation page. More information at https://haystack.deepset.ai/guides/telemetry
INFO - haystack.modeling.utils -  Using devices: CUDA:0
INFO - haystack.modeling.utils -  Number of GPUs: 1


In [ ]:
# Let's first get some documents that we want to query
# Here: 517 Wikipedia articles for Game of Thrones

s3_url = "https://www.kaspersky.com/resource-center/definitions/what-is-cyber-security"
fetch_archive_from_http(url=s3_url, output_dir=doc_dir)

# convert files to dicts containing documents that can be indexed to our datastore
# You can optionally supply a cleaning function that is applied to each doc (e.g. to remove footers)
# It must take a str as input, and return a str.
docs = convert_files_to_docs(dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True)

# We now have a list of dictionaries that we can write to our document store.
# If your texts come from a different source (e.g. a DB), you can of course skip convert_files_to_dicts() and create the dictionaries yourself.
# The default format here is: {"name": "<some-document-name>, "text": "<the-actual-text>"}

# Let's have a look at the first 3 entries:
print(docs[:3])
# Now, let's write the docs to our DB.
document_store.write_documents(docs)

INFO - haystack.utils.import_utils -  Found data stored in `data/tutorial3`. Delete this first if you really want to fetch new data.
INFO - haystack.utils.preprocessing -  Converting data/tutorial3/341_Ned_Stark.txt
INFO - haystack.utils.preprocessing -  Converting data/tutorial3/85_Game_of_Thrones__Seven_Kingdoms.txt
INFO - haystack.utils.preprocessing -  Converting data/tutorial3/444_Cripples__Bastards__and_Broken_Things.txt
INFO - haystack.utils.preprocessing -  Converting data/tutorial3/21_The_Bear_and_the_Maiden_Fair__song_.txt
INFO - haystack.utils.preprocessing -  Converting data/tutorial3/408_The_Last_of_the_Starks.txt
INFO - haystack.utils.preprocessing -  Converting data/tutorial3/198_A_Clash_of_Kings.txt
INFO - haystack.utils.preprocessing -  Converting data/tutorial3/262_Gilly__character_.txt
INFO - haystack.utils.preprocessing -  Converting data/tutorial3/443_The_Kingsroad.txt
INFO - haystack.utils.preprocessing -  Converting data/tutorial3/209_Mockingbird__Game_of_Thrones

[<Document: {'content': '\'\'\'Eddard\'\'\' "\'\'\'Ned\'\'\'" \'\'\'Stark\'\'\' is a fictional character in the 1996 fantasy novel \'\'A Game of Thrones\'\' by George R. R. Martin, and \'\'Game of Thrones\'\', HBO\'s adaptation of Martin\'s \'\'A Song of Ice and Fire\'\' series. In the storyline, Ned is the lord of Winterfell, an ancient fortress in the North of the fictional continent of Westeros. Though the character is established as the main character in the novel and the first season of the TV adaptation, a plot twist involving Ned at the end of the novel and the end of the first season shocked both readers of the book and viewers of the TV series.\nNed is portrayed by Sean Bean in the first season of \'\'Game of Thrones\'\', as a child by Sebastian Croft in the sixth season and as a young adult by Robert Aramayo in the sixth and seventh seasons. Bean was nominated for a Saturn Award for Best Actor on Television and a Scream Award for Best Fantasy Actor for the role. He and the re

INFO - haystack.document_stores.base -  Duplicate Documents: Document with id '9e9a3181b6bc168b4a25429b641e8c86' already exists in index 'document'
INFO - haystack.document_stores.base -  Duplicate Documents: Document with id '7ed12f389f7f085bb30c7d00abd26f81' already exists in index 'document'
INFO - haystack.document_stores.base -  Duplicate Documents: Document with id '9e9a3181b6bc168b4a25429b641e8c86' already exists in index 'document'
INFO - haystack.document_stores.base -  Duplicate Documents: Document with id '7ed12f389f7f085bb30c7d00abd26f81' already exists in index 'document'
INFO - haystack.document_stores.base -  Duplicate Documents: Document with id 'c8b51f62e0fccac8361c4464cc2c8f70' already exists in index 'document'
INFO - haystack.document_stores.base -  Duplicate Documents: Document with id 'c8b51f62e0fccac8361c4464cc2c8f70' already exists in index 'document'
INFO - haystack.document_stores.base -  Duplicate Documents: Document with id '9e9a3181b6bc168b4a25429b641e8c86'

In [ ]:
# An in-memory TfidfRetriever based on Pandas dataframes

from haystack.nodes import TfidfRetriever

retriever = TfidfRetriever(document_store=document_store)

INFO - haystack.nodes.retriever.sparse -  Found 2357 candidate paragraphs from 2357 docs in DB


In [ ]:
# Load a  local model or any of the QA models on
# Hugging Face's model hub (https://huggingface.co/models)

reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)


INFO - haystack.modeling.utils -  Using devices: CUDA:0
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find deepset/roberta-base-squad2 locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...
INFO - haystack.modeling.model.language_model -  Loaded deepset/roberta-base-squad2
INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.infer -  Got ya 2 parallel workers to do inference ...
INFO - haystack.modeling.infer -   0     0  
INFO - haystack.modeling.infer -  /w\   /w\ 
INFO - haystack.modeling.infer -  /'\   / \ 


In [ ]:
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)

In [ ]:
# You can configure how many candidates the reader and retriever shall return
# The higher top_k for retriever, the better (but also the slower) your answers.
prediction = pipe.run(
    query="what is cyber security?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]/usr/local/lib/python3.7/dist-packages/haystack/modeling/model/prediction_head.py:483: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  start_indices = flat_sorted_indices // max_seq_len
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 43.17 Batches/s]


In [ ]:
# Now you can either print the object directly...
from pprint import pprint

print(pprint(prediction))

{'answers': [<Answer {'answer': 'Eddard', 'type': 'extractive', 'score': 0.9919579923152924, 'context': "s Nymeria after a legendary warrior queen. She travels with her father, Eddard, to King's Landing when he is made Hand of the King. Before she leaves,", 'offsets_in_document': [{'start': 147, 'end': 153}], 'offsets_in_context': [{'start': 72, 'end': 78}], 'document_id': 'ba2a8e87ddd95e380bec55983ee7d55f', 'meta': {'name': '43_Arya_Stark.txt'}}>,
             <Answer {'answer': 'Ned', 'type': 'extractive', 'score': 0.9767242968082428, 'context': "\n====Season 1====\nArya accompanies her father Ned and her sister Sansa to King's Landing. Before their departure, Arya's half-brother Jon Snow gifts A", 'offsets_in_document': [{'start': 46, 'end': 49}], 'offsets_in_context': [{'start': 46, 'end': 49}], 'document_id': '180c2a6b36369712b361a80842e79356', 'meta': {'name': '43_Arya_Stark.txt'}}>,
             <Answer {'answer': 'Robert Baratheon', 'type': 'extractive', 'score': 0.940885454416

In [ ]:
# ...or use a util to simplify the output
# Change `minimum` to `medium` or `all` to raise the level of detail
print_answers(prediction, details="minimum")


Query: what is cyber security?
Answers:
[   {   'answer': 'No One',
        'context': 'isode was written by series co-creators David Benioff and '
                   'D. B. Weiss.\n'
                   '"No One" was written by the series\' creators David '
                   'Benioff and D. B. Weiss.\n'
                   'In '},
    {   'answer': 'Weiss',
        'context': ' what I once was."\n'
                   'In the conclusion of the "Inside the Episode" segment '
                   'Weiss spoke about Arya\'s story, saying "Arya is in '
                   "danger, she's got an open "},
    {   'answer': '===Description===',
        'context': '\n'
                   '===Description===\n'
                   "In ''A Game of Thrones'' (1996), Jaime is introduced as "
                   'one of the Kingsguard, the royal security detail, and the '
                   'son of the wealth'},
    {   'answer': 'security breach',
        'context': 'e "for internal viewing only" waterma